## 10fps로 바꾸는 것까지 진행 완료

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time

In [7]:
mp_hands = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

os.makedirs('Datasets/', exist_ok=True)

In [8]:
study_frame=300 # 10fps 기준으로 10초
FPS = 30
prev_time = 0
hand="Right" 
pause_time=5
gesture_list = ['NONE', 'URF', 'UMF', 'ULF', 'DLF', 'DMF', 'DRF', 'URB', 'UMB', 'ULB', 'DLB', 'DMB', 'DRB','URO', 'ULO', 'DLO', 'DRO']
session=1
gesture=gesture_list[1]

In [1]:
position

NameError: name 'position' is not defined

In [5]:
print('::Brushing teeth  Procedure::')
print(hand, end='\t')
for gesture in gesture_list:
    if gesture==gesture_list[-1]:
        print(gesture, end='')
    else:
        print(gesture, end='')
        print('->', end='')
print()

for gesture in gesture_list:
    cap = cv2.VideoCapture(0)

    position = np.zeros((1,21,3))
    res=np.zeros((1,21,3))

    with mp_hands.Hands(model_complexity=1,
                        max_num_hands=1,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5) as hands:



        print('::{} {} Ready.... '.format(hand,gesture))
        cnt=pause_time

        for i in [1000]*pause_time:
            print(cnt)
            cv2.waitKey(i)
            cnt-=1

        print('Start')

        while len(position) <= study_frame:

            success, frame = cap.read()
            current_time = time.time() - prev_time
            
            if not success:
                break

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                              mp_drawing_styles.get_default_hand_landmarks_style(),
                                              mp_drawing_styles.get_default_hand_connections_style())

            frame_vector_x = []
            frame_vector_y = []
            frame_vector_z = []
            if results.multi_hand_landmarks:
                for point in results.multi_hand_landmarks[0].landmark:
                    frame_vector_x.append(point.x)
                    frame_vector_y.append(point.y)
                    frame_vector_z.append(point.z)
                frame_array = np.vstack((frame_vector_x,frame_vector_y,frame_vector_z)).reshape(1,21,3)

                position = np.vstack((position,frame_array))

            if (success is True) and (current_time > 1./FPS):
                prev_time = time.time()
                cv2.putText(frame,'Dataset Count : %d' % (position.shape[0]-1),(20,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
                cv2.imshow('frame', frame)

                if cv2.waitKey(1) == ord('q'):
                    break

        position=position[1:]

        np.save('Datasets/%s_%s_p_%d' % (hand,gesture,session), position)
        cv2.waitKey(1000)


    cap.release()
    cv2.destroyAllWindows()
    print('Finished')

::Brushing teeth  Procedure::
Right	NONE->URF->UMF->ULF->DLF->DMF->DRF->URB->UMB->ULB->DLB->DMB->DRB->URO->ULO->DLO->DRO
::Right NONE Ready.... 
5
4
3
2
1
Start
Finished
::Right URF Ready.... 
5
4
3
2
1
Start


KeyboardInterrupt: 